### Convex hull trick
- 어떤 점화식이 다음 조건을 만족하는 점화식일 때 최적화할 수 있는 방법
  - $\displaystyle dp[i] = \min_{j<i}(dp[j] + a[i] \cdot b[j])$
  - $b[j]$ 는 단조 감소 ($b[j] \geq b[j+1]$)
- $\Omicron(N^2)$ 을 $\Omicron(N \log N)$ 으로 줄일 수 있다. 
  - 추가로 $a[i]$ 가 단조 증가일 때는 $\Omicron(N)$ 으로 줄일 수 있다.

### 원리
- 기본적으로 함수 개형을 통한 최적화이다.
- 점화식을 함수로 정의한다.
  - $\displaystyle f(i) = \min_{j < i}(a[i] \cdot b[j] + dp[j])$
-  $b[j]$ 가 단조 감소이므로 $a[i]$ 를 $x$ 로 잡는다면 다음과 같은 꼴의 일차함수로 바뀌게 된다.
  - $\displaystyle f(x = a[i]) = b[j] + x \cdot dp[j]$
- 어떤 문제가 이러한 $x$ 가 주어졌을 때 여러개의 $f(x)$ 중 최솟값을 구하는 문제라고 해보자. \
일차함수 여러개가 있다고 하자.
  - ![함수 개형1](./assets/CHT-1.png)
- 이 때 최솟값이 존재하는 위치는 다음과 같다.
  - ![함수 개형2](./assets/CHT-2.png)
- 최솟값이 존재하는 값들이 Convex Hull 모양이기 때문에 Convex Hull Trick 이라고 부른다.

### 증명
- 몰?

### $\Omicron (N \log N)$ Convex hull trick
- 이분 탐색을 통해 지금까지 추가한 일차함수 중 최솟값을 이분탐색으로 찾는 방법이다.
1. 초기값 설정: DP[0] = 0
2. 최솟값을 구할 $n$ 개의 일차함수에 대하여 다음 과정을 반복한다. $i = 1 \to n$ 에 대해
    - $i$ 번째 일차함수를 추가한다. 즉 기울기가 b[i], 절편 = dp[i]
    - 현재 들어간 선분 중 최솟값을 찾는다 (dp[i])

### $\Omicron(N)$ Convex hull trick
- 위의 방법에서 DP[i] 를 구하는 과정이 $\Omicron(1)$ 이 될 수 있다면 $\Omicron(N)$ 으로 줄일 수 있다.
- $a$, 즉 기울기가 단조증가 할 경우 $i$ 가 증가함에 따라 $x = a[i]$ 도 증가하게 된다. 
- 빨간색(A), 노란색(B)이 있는 상황에서 초록색(C)의 함수를 넣는다고 가정해보자.
  - ![O(N) 1](./assets/CHT-3.png)
- $A \cap B$ (파란색) 이 $B \cap C$ (검정색) 보다 왼쪽에 있다는 것은 $B$ 가 쓸모있다는 것을 의미한다.
  - ![O(N) 2](./assets/CHT-4.png)
- 반면 $A \cap B$ 가 $B \cap C$ 보다 오른쪽에 있다면 각각은 $A \cap C$ 보다 높아지게 되므로 $B$ 는 쓸모가 없게된다. \
이 경우 B를 제거하고 C를 추가하면 된다.

### 구현
- 직선의 방정식은 $A[i] \cdot x + B[i]$
- 실수 구현도 문제 없이 작동한다.
- min query이다. `if self.B[I[-1]] < self.B[i]:` 의 부호를 반대로 바꿔서 max query로 바꿀 수 있다.
- 코드 주석은 14751(Leftmost Segment) 참고

In [ ]:
from bisect import bisect_left
class CHT :
  def __init__(self, A, B, EPS=1e-7) :
    self.A = A
    self.B = B
    self.EPS = EPS
    self.build()
  
  def intersect(self, i, j) :
    return (self.B[j] - self.B[i]) / (self.A[i] - self.A[j])

  def build(self) :
    I, X = [], []    
    order = sorted(range(len(self.A)), key=self.A.__getitem__, reverse=True)
    for i in order:
      while True:
        if not I:
          I.append(i)
          break
        elif (self.A[I[-1]] - self.A[i]) < self.EPS :
          if self.B[I[-1]] < self.B[i] :
            break
          I.pop()
          if X: X.pop()
        else:
          x = self.intersect(i, I[-1])
          if X and x <= X[-1] :
            I.pop()
            X.pop()
          else:
            I.append(i)
            X.append(x)
            break
    self.I = I
    self.X = X

  def query(self, x) :
    i = self.I[bisect_left(self.X, x + self.EPS)]
    return self.A[i] * x + self.B[i]

- 정점을 추가하면서 쿼리하는 템플릿

In [ ]:
from bisect import bisect_left
class CHT :
  def __init__(self, A=[], B=[], EPS=1e-7) :
    self.A = A
    self.B = B
    self.I = []
    self.X = []
    self.i = 0
    self.EPS = EPS
    self.build()
  
  def intersect(self, i, j) :
    return (self.B[j] - self.B[i]) / (self.A[i] - self.A[j])

  def add(self, a, b, i=None) : #Amortized O(1)
    self.A.append(a)
    self.B.append(b)
    I, X = self.I, self.X
    if i is None :
      i = self.i
      self.i += 1

    while True:
      if not I:
        I.append(i)
        break
      elif (self.A[I[-1]] - self.A[i]) < self.EPS :
        if self.B[I[-1]] < self.B[i] :
          break
        I.pop()
        if X: X.pop()
      else:
        x = self.intersect(i, I[-1])
        if X and x <= X[-1] :
          I.pop()
          X.pop()
        else:
          I.append(i)
          X.append(x)
          break

  def build(self) :
    I, X = [], []    
    order = sorted(range(len(self.A)), key=self.A.__getitem__, reverse=True)
    for i in order:
      self.add(self.A[i], self.B[i], i)
    self.I = I
    self.X = X

  def query(self, x) : #O(logN)
    i = self.I[bisect_left(self.X, x + self.EPS)]
    return self.A[i] * x + self.B[i]